In [19]:
# !pip install chardet

import chardet

with open("datasets/hotels.csv", "rb") as f:
    raw_data = f.read(100000)  # Read first 100,000 bytes (adjust if needed)
    result = chardet.detect(raw_data)

print("Detected encoding:", result["encoding"])

Detected encoding: UTF-8-SIG


In [20]:
import pandas as pd

df = pd.read_csv("datasets/hotels.csv", encoding="UTF-8-SIG", encoding_errors="replace") 

print(df.head(5))

                                            Page_URL  page  \
0  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
1  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
2  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
3  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
4  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   

                                         name  \
0                          Ace Hotel New York   
1                       Moxy NYC East Village   
2                           The Knickerbocker   
3         Hyatt Centric Times Square New York   
4                    Conrad New York Downtown   

                                           image_url        price  \
0  https://media-cdn.tripadvisor.com/media/photo-...   60 718 PKR   
1  https://media-cdn.tripadvisor.com/media/photo-...   69 512 PKR   
2  https://media-cdn.tripadvisor.com/media/photo-...  146 591 PKR   
3  https://media-cdn.tripadvisor.com/media/photo-...   9

In [21]:
# !pip install tabulate

import pandas as pd

df = pd.read_csv("datasets/hotels.csv", encoding="UTF-8-SIG", encoding_errors="replace") 
print(df.to_markdown())

|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                                    | price       | expedia     | agoda_com   | hotels      | number_reviews   |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|:------------|:------------|:------------|:------------|:-----------------|
|   0 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Ace Hotel New York                                       | https://media-cdn.tripadvisor.com/media/photo-s/1a/c4/7b/a2/lobby.jpg                      

In [22]:
# Count total missing values
missing_values = df.isna().sum().sum()
print(f"Total missing values: {missing_values}")

# Print only rows with at least one NaN value
null_rows = df[df.isna().any(axis=1)]
print(null_rows.to_markdown())  # Displays all rows with missing values

Total missing values: 108
|     | Page_URL                                                                                             |   page |   name |   image_url |   price | expedia   | agoda_com   | hotels   |   number_reviews |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|-------:|------------:|--------:|:----------|:------------|:---------|-----------------:|
|   7 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 |    nan |         nan |     nan | /         | /           | /        |              nan |
|  18 | https://www.tripadvisor.fr/Hotels-g295424-Dubai_Emirate_of_Dubai-Hotels.html                         |      1 |    nan |         nan |     nan | /         | /           | /        |              nan |
|  29 | https://www.tripadvisor.fr/Hotels-g295424-oa30-Dubai_Emirate_of_Dubai-Hotels.html                    |      2 |    nan |         n

In [23]:
print(df.dtypes)

Page_URL          object
page               int64
name              object
image_url         object
price             object
expedia           object
agoda_com         object
hotels            object
number_reviews    object
dtype: object


In [24]:
# df["number_reviews"] = df["number_reviews"].str.replace("avis", "", regex=True).str.strip() 
# df["number_reviews"] = pd.to_numeric(df["number_reviews"], errors="coerce").astype("Int64")  

# Remove non-breaking space (U+202F) and 'avis' from the 'number_reviews' column
df["number_reviews"] = df["number_reviews"].astype(str)  # Ensure the column is a string

# Remove 'avis' and non-breaking space (U+202F)
df["number_reviews"] = df["number_reviews"].str.replace("\u202F", "", regex=True).str.replace("avis", "", regex=True).str.strip()

# Convert the cleaned column to numeric (handles any remaining non-numeric values as NaN)
df["number_reviews"] = pd.to_numeric(df["number_reviews"], errors='coerce')

print(df.to_markdown())


|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                                    | price       | expedia     | agoda_com   | hotels      |   number_reviews |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|:------------|:------------|:------------|:------------|-----------------:|
|   0 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Ace Hotel New York                                       | https://media-cdn.tripadvisor.com/media/photo-s/1a/c4/7b/a2/lobby.jpg                      

# Extract the city name from the Page URL

In [25]:
import re
import pandas as pd
import geonamescache
import numpy as np

# Load geonamescache database
gc = geonamescache.GeonamesCache()
city_names = set(v["name"] for v in gc.get_cities().values())  # Store cities in a set for fast lookup

def extract_city(url):
    """Extract city name from URL using regex and match with known city names."""
    match = re.search(r'Hotels-g\d+.*?-([A-Za-z_]+)-Hotels\.html', url)
    if match:
        location = match.group(1).replace("_", " ")  # Normalize location name
        words = location.split()
        
        # Find the longest matching city name
        for i in range(len(words), 0, -1):
            city_candidate = " ".join(words[:i])
            if city_candidate in city_names:
                return city_candidate  # Return the first valid city found
        
    return np.nan  # Return NaN if no match found

# Apply function to extract city
df["City"] = df["Page_URL"].apply(extract_city)

print(df.to_markdown())



|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                                    | price       | expedia     | agoda_com   | hotels      |   number_reviews | City          |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|:------------|:------------|:------------|:------------|-----------------:|:--------------|
|   0 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Ace Hotel New York                                       | https://media-cdn.tripadvisor.com/media/photo-s/1a/c4/7b/a2

In [26]:
print(sorted(df["City"].dropna().unique()))

['Agadir', 'Angers', 'Berlin', 'Budapest', 'Casablanca', 'Dubai', 'Hammamet', 'Istanbul', 'Las Vegas', 'Lisbon', 'London', 'Milan', 'New York City', 'Rome', 'Venice']


In [27]:
# Define a mapping of cities to countries
city_to_country = {
    "Agadir": "Morocco",
    "Angers": "France",
    "Berlin": "Germany",
    "Budapest": "Hungary",
    "Casablanca": "Morocco",
    "Dubai": "United Arab Emirates",
    "Hammamet": "Tunisia",
    "Istanbul": "Turkey",
    "Las Vegas": "United States",
    "Lisbon": "Portugal",
    "London": "United Kingdom",
    "Milan": "Italy",
    "New York City": "United States",
    "Rome": "Italy",
    "Venice": "Italy"
}

# Map the City column to the Country column
df["Country"] = df["City"].map(city_to_country)

# Display the updated dataframe
print(df.head())


                                            Page_URL  page  \
0  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
1  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
2  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
3  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   
4  https://www.tripadvisor.fr/Hotels-g60763-oa30-...     2   

                                         name  \
0                          Ace Hotel New York   
1                       Moxy NYC East Village   
2                           The Knickerbocker   
3         Hyatt Centric Times Square New York   
4                    Conrad New York Downtown   

                                           image_url        price  \
0  https://media-cdn.tripadvisor.com/media/photo-...   60 718 PKR   
1  https://media-cdn.tripadvisor.com/media/photo-...   69 512 PKR   
2  https://media-cdn.tripadvisor.com/media/photo-...  146 591 PKR   
3  https://media-cdn.tripadvisor.com/media/photo-...   9

In [28]:
# !pip install forex_python.converter
# !python.exe -m pip install --upgrade pip

In [29]:
import numpy as np 

# Function to clean and convert currency values
def clean_currency(value):
    if value == '/' or pd.isna(value):  # Convert '/' to NaN
        return np.nan
    value = value.replace('PKR', '').strip()  # Remove 'PKR'
    value = value.replace(' ', '').replace(',', '')  # Remove non-breaking spaces and commas
    return float(value)  # Convert to float

# Apply function to relevant columns
columns_to_clean = ['price', 'agoda_com', 'expedia', 'hotels']
df[columns_to_clean] = df[columns_to_clean].applymap(clean_currency)

C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_3796\1088260787.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_clean] = df[columns_to_clean].applymap(clean_currency)


In [30]:
# Count total missing values
missing_values = df.isna().sum().sum()
print(f"Total missing values: {missing_values}")

# Print only rows with at least one NaN value
null_rows = df[df.isna().any(axis=1)]
print(null_rows.to_markdown())  

Total missing values: 466
|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                      |   price |   expedia |   agoda_com |   hotels |   number_reviews | City          | Country              |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-----------------------------------------------------------------------------------------------|--------:|----------:|------------:|---------:|-----------------:|:--------------|:---------------------|
|   1 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Moxy NYC East Village                                    | https://media-cdn.tripadvisor.com

In [31]:
# Exchange rate: 1 PKR = 0.0035 EUR
exchange_rate = 0.0035

# Columns to convert
columns_to_convert = ['price', 'agoda_com', 'expedia', 'hotels']

# Apply conversion only to the specified columns
df[columns_to_convert] = (df[columns_to_convert] * exchange_rate).round(2)


In [32]:
import numpy as np

# Assuming the column is called 'number_reviews'
df["number_reviews"] = df["number_reviews"].astype(str)  # Ensure it's a string

# Remove the 'avis' string and non-breaking spaces
df["number_reviews"] = df["number_reviews"].str.replace("avis", "", regex=True).str.replace("\u202F", "", regex=True).str.strip()


df["number_reviews"] = df["number_reviews"].replace('nan', np.nan)

# Convert the column to numeric, turning invalid strings (like '<NA>' or non-numeric strings) into NaN
df["number_reviews"] = pd.to_numeric(df["number_reviews"], errors='coerce')

# Check the dtypes to confirm
print(df.dtypes)

Page_URL           object
page                int64
name               object
image_url          object
price             float64
expedia           float64
agoda_com         float64
hotels            float64
number_reviews    float64
City               object
Country            object
dtype: object


In [33]:
print(df.to_markdown())

|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                                    |   price |   expedia |   agoda_com |   hotels |   number_reviews | City          | Country              |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------|--------:|----------:|------------:|---------:|-----------------:|:--------------|:---------------------|
|   0 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Ace Hotel New York                                       | https://media-cdn.tripadvisor.c

# I removed the nan values from column 'name' because how can we know where it is if the name doesn't exist

In [34]:
df = df.dropna(subset=["name"])

In [35]:
# Count total missing values
missing_values = df.isna().sum().sum()
print(f"Total missing values: {missing_values}")

# Print only rows with at least one NaN value
null_rows = df[df.isna().any(axis=1)]
print(null_rows.to_markdown())  

Total missing values: 273
|     | Page_URL                                                                                             |   page | name                                                     | image_url                                                                                      |   price |   expedia |   agoda_com |   hotels |   number_reviews | City          | Country              |
|----:|:-----------------------------------------------------------------------------------------------------|-------:|:---------------------------------------------------------|:-----------------------------------------------------------------------------------------------|--------:|----------:|------------:|---------:|-----------------:|:--------------|:---------------------|
|   1 | https://www.tripadvisor.fr/Hotels-g60763-oa30-New_York_City_New_York-Hotels.html                     |      2 | Moxy NYC East Village                                    | https://media-cdn.tripadvisor.com

In [36]:
import os

# Define the folder path
folder_path = "converted_datasets"

# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)

# Define the full file path
file_path = os.path.join(folder_path, "hotels_new.csv")

# Save the DataFrame as a CSV file
df.to_csv(file_path, index=False, encoding='utf-8')

print(f"CSV file saved successfully at: {file_path}")


CSV file saved successfully at: converted_datasets\hotels_new.csv
